# BERTopic

In [50]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
import re

import os

In [51]:
with open('/Volumes/Drive/GitHub/DaVinciCodeTheTrackOfRobertLangdon/models/CoRef-TheDaVinciCode.txt', 'r') as f:
    data = f.read()
    data = data.replace("\n", " ")
    data = data.replace("\t", " ")
    data = data.replace("\r", " ")
    data = data.replace("  ", " ")
    data = data.replace("   ", " ")
    data = data.replace("    ", " ")
    data = data.replace("     ", " ")
    ' '.join(data.split())
    data = re.sub('/\s+/', ' ', data)
f.close()
# paragraphs = data.split("CHAPTER")

<>:11: DeprecationWarning: invalid escape sequence \s
<>:11: DeprecationWarning: invalid escape sequence \s
/var/folders/0d/97nbyjxx2cx9d8_vlthd88qh0000gn/T/ipykernel_1664/3342396829.py:11: DeprecationWarning: invalid escape sequence \s
  data = re.sub('/\s+/', ' ', data)


In [52]:
temp = data.split(".")
temp_ = [x for x in temp if x != ['']]
# merge by index of 5 sentences
sentences = [temp_[i:i+5] for i in range(0, len(temp_), 5)]
# merge all subindex of sentences into one string
sentences = [' '.join(x) for x in sentences]
paragraphs = data.split("CHAPTER")

In [53]:
# Fine-tune your topic representations
representation_model = KeyBERTInspired()
topic_model = BERTopic(language="english",representation_model=representation_model, calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(paragraphs)

Batches: 100%|██████████| 4/4 [00:03<00:00,  1.21it/s]
2023-05-27 14:05:07,056 - BERTopic - Transformed documents to Embeddings
2023-05-27 14:05:08,388 - BERTopic - Reduced dimensionality
2023-05-27 14:05:08,426 - BERTopic - Clustered reduced embeddings


In [54]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,6,-1_depository_bank_zurich_hangar
1,0,78,0_langdon_langdons_priory_sophie
2,1,25,1_silas_silass_opus_saintsulpice


In [55]:
freq

,Topic,Count,Name
0,-1,6,-1_depository_bank_zurich_hangar
1,0,78,0_langdon_langdons_priory_sophie
2,1,25,1_silas_silass_opus_saintsulpice


In [63]:
df = topic_model.get_document_info(paragraphs)
# df only for topic 0
df = df[df['Topic']==1]
# converge all document in df as one single string
text = ' '.join(df['Document'].tolist()).lower()
# count all the occurences of langdon in the text
print('Langdon is present %d times in topic 0' % (text.count('vatican')))

Langdon is present 50 times in topic 0


In [33]:
df = topic_model.get_document_info(paragraphs)
# df only for topic 1
df = df[df['Topic']==1]
# converge all document in df as one single string
text = ' '.join(df['Document'].tolist()).lower()
# count all the occurences of langdon in the text
print('Langdon is present %d times in topic 1' % (text.count('langdon')))

Langdon is present 99 times in topic 1


In [34]:
df = topic_model.get_document_info(paragraphs)
# df only for topic 0
df = df[df['Topic']==0]
# converge all document in df as one single string
text = ' '.join(df['Document'].tolist()).lower()
# count all the occurences of langdon in the text
print('Silas is present %d times in topic 0' % (text.count('silas')))

Silas is present 130 times in topic 0


In [35]:
df = topic_model.get_document_info(paragraphs)
# df only for topic 1
df = df[df['Topic']==1]
# converge all document in df as one single string
text = ' '.join(df['Document'].tolist()).lower()
# count all the occurences of langdon in the text
print('Silas is present %d times in topic 1' % (text.count('silas')))

Silas is present 782 times in topic 1


In [36]:
topics, probs = topic_model.fit_transform(sentences)

Batches: 100%|██████████| 79/79 [00:25<00:00,  3.04it/s]
2023-05-26 15:04:09,336 - BERTopic - Transformed documents to Embeddings
2023-05-26 15:04:14,033 - BERTopic - Reduced dimensionality
2023-05-26 15:04:14,330 - BERTopic - Clustered reduced embeddings


In [37]:
topic_model.visualize_topics()

In [38]:
topic_model.visualize_heatmap()

In [21]:
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
from gensim.models.coherencemodel import CoherenceModel

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kristianvankuijk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(words):
    stop_words = stopwords.words('english')
    new_word = []
    flag = False
    for word in words:
        for stop_word in stop_words:
            if stop_word in word:
                flag = True
                break
        if not flag:
            new_word.append(word)
        flag = False
    return new_word
    # return [word for word in words if word not in stop_words]

In [47]:
stop_words = stopwords.words('english')
sentences = data.split(".")

words = sent_to_words(sentences)
# Remove stop words
words = remove_stopwords(words)

id2word = corpora.Dictionary(words)

In [64]:
topics = topic_model.get_topics()

# Create a dictionary from the words
id2word = corpora.Dictionary(words)

# Create a corpus for coherence evaluation
corpus = [id2word.doc2bow(w) for w in words]

topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]


coherence_model = CoherenceModel(topics=topic_words, texts=words, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_score = coherence_model.get_coherence()

print("Coherence Score:", coherence_score)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# LDA

In [49]:
# https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

stop_words = stopwords.words('english')
sentences = data.split(".")

words = sent_to_words(sentences)
# Remove stop words
words = remove_stopwords(words)

id2word = corpora.Dictionary(words)

# Term Document Frequency
corpus = [id2word.doc2bow(word) for word in words]
num_topics = 5

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       passes=20,
                                       iterations=5000)

# Print the keywords in 5 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

# Visualize the topics
pyLDAvis.enable_notebook()

if not os.path.exists('LDA'):
  os.mkdir('LDA')
  
LDAvis_data_filepath = os.path.join('LDA/results')

# Visualize
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
with open(LDAvis_data_filepath + '10', 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

pyLDAvis.save_html(LDAvis_prepared, 'LDA/results'+ str(10) +'.html')

LDAvis_prepared

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.159266  0.088240       1        1  26.146700
4     -0.030727 -0.110510       2        1  22.391077
1     -0.009357 -0.133933       3        1  19.692733
0      0.017000  0.115382       4        1  18.593931
3      0.182349  0.040819       5        1  13.175559, topic_info=        Term        Freq       Total Category  logprob  loglift
274   sophie  162.000000  162.000000  Default  30.0000  30.0000
14       yes   22.000000   22.000000  Default  29.0000  29.0000
20   langdon  195.000000  195.000000  Default  28.0000  28.0000
37    paused   32.000000   32.000000  Default  27.0000  27.0000
46    looked   27.000000   27.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
178     eyes    2.516217   13.818296   Topic5  -5.3184   0.3236
131    fache    3.034339   35.908602   Topic5  -5.1312  -0.4442
301   collet    1.604627   13.887615   Topic5  -5.7683  -0.1313
371   simple    1.525819    2.855310   Topic5  -5.8186   1.4002
242       un    1.527912    5.058644   Topic5  -5.8173   0.8296

[256 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
192       2  0.522873     agent
192       3  0.209149     agent
192       4  0.209149     agent
956       5  0.947979    amazed
632       1  0.976378     andre
...     ...       ...       ...
789       4  0.545886    winked
419       3  0.686383     woman
765       3  0.915187  wretched
14        2  0.087245       yes
14        5  0.916068       yes

[307 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 1, 4])

In [26]:
cm = CoherenceModel(model=lda_model, texts=words, corpus=corpus, coherence='c_v')
coherence = cm.get_coherence()  # get coherence value
print('coherence:', coherence)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 